## combining chromosome SNP data per sample into WES SNP data
### also add EB data if available to filter out bad SNPs

In [1]:
# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = os.path.join(home,"Dropbox/Icke/Work/somVar/testdata")
tooldata = os.path.join(home, "Dropbox/Icke/Work/somVar/tooldata")
cnvdata = os.path.join(tooldata, "myCNVdata")
shell_path = "../shell"
static_path = os.path.join(home, "Dropbox/Icke/Work/static")
output_path = os.path.join(cnvdata, "output")

cnv_path = os.path.join(cnvdata, "cnv")

In [2]:
cnv_path

'/Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv'

In [3]:
test = pd.read_csv(f'{cnv_path}/01_A.chr3.snpEB', sep='\t').loc[:,['Chr', 'Start', 'Ref','EBscore', 'PoN-Alt']]
test[:13]
test = pd.read_csv(f'{cnv_path}/01_A.chr3.snp', sep='\t')
test[:13]

,Chr,Start,ExonPos,Ref,Depth,Alt,VAF
0,chr3,197596,50,T,112,C1,0.008929
1,chr3,197682,136,T,197,G2,0.010152
2,chr3,328224,768,A,188,T1,0.005319
3,chr3,340795,950,T,348,A1,0.002874
4,chr3,340862,1017,C,536,G1,0.001866
5,chr3,342011,1209,G,160,A1,0.006250
6,chr3,342964,1360,C,179,A1,0.005587
7,chr3,343057,1453,G,236,T1,0.004237
8,chr3,344564,1484,C,119,T60,0.504202
9,chr3,344607,1527,G,192,T1,0.005208


### combine all snp for a sample into one df with corresponding EBscores

In [6]:
# try with pd merge
import os
chrom_list = [f"chr{chrom + 1}" for chrom in range(22)] + ['chrX']


def combine_SNPdata(sample, cnv_path):
    snp_dfs = []
    file_base = os.path.join(cnv_path, sample)
    for chrom in chrom_list:
        # reading SNP
        file = f"{file_base}.{chrom}.snp"
        if not os.path.isfile(file):
            continue
        print(f"Reading SNP VAF from {chrom} of sample {sample} from {file}.")
        snp_df = pd.read_csv(file, sep='\t')
        snp_df[['Alt', 'AltDepth']] = snp_df['Alt'].str.extract(r"([AGCT])([0-9]+)")
        
        #reading snpEB
        file = f"{file_base}.{chrom}.snpEB"
        if not os.path.isfile(file):
            continue
        print(f"Reading SNP EBscores from {chrom} of sample {sample} from {file}.")
        snpEB_df = pd.read_csv(file, sep='\t').loc[:,['Chr', 'Start', 'Ref','Alt', 'EBscore', 'PoN-Alt']]
        snp_df = snp_df.merge(snpEB_df, on=['Chr', 'Start', 'Ref', 'Alt'])
        
        snp_dfs.append(snp_df)
    snp_df = pd.concat(snp_dfs)
    return snp_df.loc[:, ["Chr", "Start", "ExonPos", "Ref", "Depth", "Alt", "VAF", "EBscore", "PoN-Alt"]]

In [10]:
sample = "01_A"
snp_df = combine_SNPdata(sample, cnv_path)

Reading SNP VAF from chr1 of sample 01_A from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv/01_A.chr1.snp.
Reading SNP EBscores from chr1 of sample 01_A from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv/01_A.chr1.snpEB.
Reading SNP VAF from chr2 of sample 01_A from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv/01_A.chr2.snp.
Reading SNP EBscores from chr2 of sample 01_A from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv/01_A.chr2.snpEB.
Reading SNP VAF from chr3 of sample 01_A from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv/01_A.chr3.snp.
Reading SNP EBscores from chr3 of sample 01_A from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv/01_A.chr3.snpEB.
Reading SNP VAF from chr4 of sample 01_A from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/cnv/01_A.chr4.snp.
Reading SNP EBscores from chr4 of sample 01_A from /Users/martinscien

In [12]:
snp_df.query('VAF > 1')

,Chr,Start,ExonPos,Ref,Depth,Alt,VAF,EBscore,PoN-Alt
1426,chr1,1968667,128857,T,70,C,1.85714,0.000,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|0|...
5386,chr1,16986749,596763,C,68,C,1.05882,0.001,39|118|95|106|58|31|35|30|21|34|41|19|8|15|18|...
6045,chr1,19312149,674448,G,44,G,1.02273,0.001,9|22|38|9|6|15|16|26|8|10|4|13|6|15|3|10|1|6|1...
6432,chr1,21449920,727347,C,180,T,1.31111,0.000,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|0|...
7204,chr1,23967960,824609,C,129,A,1.03101,0.000,1|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|0|...
...,...,...,...,...,...,...,...,...,...
7028,chr22,44862611,910792,G,121,A,1.70248,0.000,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|1|...
58,chrX,312722,2009,G,208,C,1.59615,0.000,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|0|...
2631,chrX,45076679,427780,C,262,T,1.50000,0.000,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-0|0|0|0|...
6040,chrX,106849220,1172552,a,184,T,1.16304,0.022,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0-1|4|0|2|...


### merge sample coverage with Pon coverage

In [ ]:
def combine_Covdata(sample, cnv_data_path):
    
    cover_dfs = []

    for chrom in chrom_list:
        # reading sampleCoverage
        sample_cov_file = os.path.join(cnv_data_path, f"cnv/{sample}.{chrom}.cov")
        if not os.path.isfile(sample_cov_file):
            continue
        print(f"Reading coverage from {chrom} of sample {sample} from {sample_cov_file}.")
        cov_df = pd.read_csv(sample_cov_file, sep='\t', compression="gzip")
        
        
        #reading PONcoverage
        pon_cov_file = os.path.join(cnv_data_path, f"chromCov/{chrom}.filtered.csv.gz")
        if not os.path.isfile(pon_cov_file):
            continue
        print(f"Reading PON coverage of {chrom} from file {pon_cov_file}.")
        pon_df = pd.read_csv(pon_cov_file, sep='\t', compression="gzip").loc[:,['Chr', 'Pos', 'ExonPos', 'meanCov', 'medianCov', 'std']]
        # column rename
        trans_dict = {col:f"PON{col}" for col in pon_df.columns[3:]}
        pon_df = pon_df.rename(columns=trans_dict)
        # merge sample with PON coverage
        sample_df = cov_df.merge(pon_df, on=['Chr', 'Pos', 'ExonPos'], how="inner").loc[:,['Chr', 'Pos', 'ExonPos', 'Coverage','PONmeanCov', 'PONmedianCov', 'PONstd']]
        
        # normalize the coverage
        sample_df['Coverage'] = sample_df['Coverage'] / sample_df['Coverage'].mean() * 100
        cover_dfs.append(sample_df)   
    cover_df = pd.concat(cover_dfs)
    cover_df['log2ratio'] = np.log2(covere_df['Coverage'] / cover_df['PONmeanCov'])
    cover_df['rr100'] = cover_df['log2ratio'].rolling(100).mean()
    cover_df['rr200'] = cover_df['log2ratio'].rolling(200).mean()
    cover_df['rr500'] = cover_df['log2ratio'].rolling(500).mean()
    
    return cover_df

In [ ]:
sample = "01_A"
combine_Covdata(sample, cnvdata)

In [ ]:
snp_df = combine_heteroSNP("03_A")
snp_df.to_csv(f"{output_path}/03_A.snp.csv", sep='\t', index=False)
snp_df

### make a stacked "tidy version" of the coverage df for vizualisation in tidyverse

In [ ]:
def tidy_df(df):
    return df.drop(columns=['Chr', 'Pos']).set_index('ExonPos').stack().reset_index().rename(columns={'level_1':'sample', 0:'Coverage'})

tidy_df(cov_df).to_csv(f"{output_path}/PON_coverage_tidy.csv", sep='\t', index=False)

### normalize the coverage to coverage 100

In [ ]:
def normalize_coverage(cov_df):
    norm_df = cov_df.set_index(['Chr','Pos','ExonPos'])
    norm_df = norm_df / norm_df.mean() * 100
    return norm_df.reset_index()

In [ ]:
norm_df = normalize_coverage(cov_df)
norm_df

In [ ]:
tidy_df(norm_df).to_csv(f"{output_path}/PON_coverage_normtidy.csv", sep='\t', index=False)

### compute the mean of all the coverages

In [ ]:
def add_mean(norm_df):
    norm_df = norm_df.set_index(['Chr', 'Pos', 'ExonPos'])
    norm_df['meanCov'] = norm_df.mean(axis=1)
    norm_df['medianCov'] = norm_df.median(axis=1)
    norm_df['std'] = norm_df.std(axis=1)
    return norm_df.reset_index()
mean_df = add_mean(norm_df)
tidy_df(mean_df).to_csv(f"{output_path}/PON_coverage_mean.csv", sep='\t', index=False)
mean_df

### filter the coverage

In [ ]:
mean_df['std'].max()

In [ ]:
def filter_coverage(df, mincov, maxstd):
    filter_df = df.query('meanCov > @mincov and std < @maxstd')
    return filter_df

In [ ]:
filter_df = filter_coverage(mean_df, mincov=20, maxstd=50)
tidy_df(filter_df).to_csv(f"{output_path}/PON_coverage_filter.csv", sep='\t', index=False)